# <center> 2D spin glass with h field on the square lattice</center>
# <center> $H = - \sum_\limits{<i,j>} J_{i,j} S_{i}  S_{j}  -  h \sum_\limits{<i>} S_{i}$ </center>

# (1)计算Boltzmann矩阵

In [58]:
import torch,math
import numpy as np
from scipy.linalg import sqrtm
import os
import sys
import pickle
import random
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
warnings.filterwarnings("ignore")


#  S13---S14-----S15----S16
#  \      \      \      \
#  \      \      \      \
#  S9----S10-----S11----S12
#  \      \      \      \
#  \      \      \      \
#  S5-----S6-----S7-----S8
#  \      \      \      \
#  \      \      \      \
#  S1-----S2-----S3-----S4

def neighbor(nx,ny):
    nbor = np.zeros((nx*ny, 4))
    for ispin in range(nx*ny):
        iy = int((ispin)/nx)
        ix = ispin-(iy)*nx
        ixp = ix+1-int(ix/(nx-1))*nx           #右侧点, x坐标
        iyp = iy+1-int(iy/(ny-1))*ny           #上侧点, y坐标
        ixm = ix-1+int((nx-ix-1)/(nx-1))*nx    #左侧点, x坐标
        iym = iy-1+int((ny-iy-1)/(ny-1))*ny    #下侧点, y坐标
        nbor[ispin, 0] = (iy)*nx+ixp  #右邻居
        nbor[ispin, 1] = (iyp)*nx+ix  #上邻居
        nbor[ispin, 2] = (iy)*nx+ixm  #左邻居
        nbor[ispin, 3] = (iym)*nx+ix  #下邻居
        if iy == ny-1:
            nbor[ispin,1] = -1
        if ix == nx-1:
            nbor[ispin,0] = -1
        if ix == 0:
            nbor[ispin,2] = -1
        if iy == 0:
            nbor[ispin,3] = -1
    return nbor

def get_interaction( Nx,Ny, mydtype=torch.float64, mydevice=torch.device('cpu')):
    hx = [ [0]*Nx*Ny ]
    hy = [ [0]*Nx*Ny ]
    list = [1,-1]
    for i in range(0,Nx*Ny):
            hx[0][i]=random.sample(list, 1)[0]
    for i in range(0,Nx*Ny):
            hy[0][i]=random.sample(list, 1)[0]
    return hx,hy

def get_Bmatrix(beta, J, h, nbor, cen, nb_cen, mydtype=torch.float64, mydevice=torch.device('cpu')):
    # 判断分给多少磁场
    bonds = 0
    nb_bonds = 0
    for i in range(4):
        # print(int(cen),nbor[int(cen),i])
        if nbor[int(cen),i] != -1:
            bonds = bonds + 1
        if nbor[int(nb_cen),i] != -1:
            nb_bonds = nb_bonds + 1
    # 计算玻尔兹曼矩阵
    B=torch.tensor(np.array([
                                [np.exp(-beta*(-J - h/bonds - h/nb_bonds)),np.exp(-beta*( J - h/bonds + h/nb_bonds))],
                                [np.exp(-beta*( J + h/bonds - h/nb_bonds)),np.exp(-beta*(-J + h/bonds + h/nb_bonds))]
                            ]),dtype=mydtype,device=mydevice)
    return B

def get_Bmatrix_list(nx, ny, nbor, hx, hy, h):
    B_list = []
    for ispin in range(nx*ny):
        iy = int((ispin)/nx)
        ix = ispin-(iy)*nx
        B_1 = 0          #torch.tensor(np.array([[1,0],[0,1]])
        B_2 = 0          #torch.tensor(np.array([[1,0],[0,1]])
        if ix != nx-1:
            B_1 = get_Bmatrix(beta, hx[0][ispin], h, nbor, ispin, nbor[int(ispin), 0])
        if iy != ny-1:
            B_2 = get_Bmatrix(beta, hy[0][ispin], h, nbor, ispin, nbor[int(ispin), 1])
        B_list.append([B_1, B_2])
    return B_list


# 设置随机数种子
seed = 71
random.seed(seed)

# ----------------------------------- setting --------------------------------------------
Lx   = 4
Ly   = 4
h    = -0.221
beta = 0.53
chi  = 16
hx, hy = get_interaction(Lx,Ly, mydtype=torch.float64,mydevice=torch.device('cpu'))  # 1: 铁磁
# for i in range(Lx*Ly):
#     hx[0][i] = 0
#     hy[0][i] = 0
# ----------------------------------- save rand hx, hy -----------------------------------
np.savetxt('hxx.txt', hx)
np.savetxt('hyy.txt', hy)

# ----------------------------------- calculate Boltzmann matrix -------------------------
nbor = neighbor(Lx, Ly)
B_list = get_Bmatrix_list(Lx, Ly, nbor, hx, hy, h)

# ----------------------------------- print ----------------------------------------------
print('hx = ', hx)
print('hy = ', hy)
# display('B_list = ',B_list)

hx =  [[-1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1]]
hy =  [[1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1]]


# (2)计算张量元，并给出张量网络
* Indexing of tensors are clock-wise
<img src="./asset/image.png" width="500">

In [59]:
import torch
from numpy import linalg as la
from scipy.linalg import sqrtm
mydtype=torch.float64;
mydevice=torch.device('cpu');

def calculate_tensor( B_list, Lx, Ly, cen, nbor, mydtype=torch.float64, mydevice=torch.device('cpu')):
    # east
    if nbor[cen,0]!=-1:
        B      = B_list[cen][0]
        U,s,VT = la.svd(B)
        Sigma  = np.zeros(np.shape(B))
        Sigma[:len(s),:len(s)] = np.diag(s)
        right = U @ np.sqrt(Sigma)
        right = torch.from_numpy(right)
    # north
    if nbor[cen,1]!=-1:
        B      = B_list[cen][1]
        U,s,VT = la.svd(B)
        Sigma  = np.zeros(np.shape(B))
        Sigma[:len(s),:len(s)] = np.diag(s)
        up = U @ np.sqrt(Sigma)
        up = torch.from_numpy(up)
    # west    
    if nbor[cen,2]!=-1:
        B      = B_list[int(nbor[cen,2])][0]
        U,s,VT = la.svd(B)
        Sigma  = np.zeros(np.shape(B))
        Sigma[:len(s),:len(s)] = np.diag(s)
        left = np.sqrt(Sigma) @ VT
        left = torch.from_numpy(left)
    # south    
    if nbor[cen,3]!=-1:
        B      = B_list[int(nbor[cen,3])][1]
        U,s,VT = la.svd(B)
        Sigma  = np.zeros(np.shape(B))
        Sigma[:len(s),:len(s)] = np.diag(s)
        down = np.sqrt(Sigma) @ VT
        down = torch.from_numpy(down)

    # four bonds
    if nbor[cen,0] != -1  and  nbor[cen,1] != -1  and  nbor[cen,2] != -1  and  nbor[cen,3] != -1:   
        T = torch.einsum('ia,ib,ci,di->abcd',right,up,left,down)
    # three bonds
    if nbor[cen,0] == -1  and  nbor[cen,1] != -1  and  nbor[cen,2] != -1  and  nbor[cen,3] != -1:   
        T = torch.einsum('bi,ic,di->cbd',left,up,down)
    if nbor[cen,0] != -1  and  nbor[cen,1] == -1  and  nbor[cen,2] != -1  and  nbor[cen,3] != -1:
        T = torch.einsum('ia,bi,di->abd',right,left,down)
    if nbor[cen,0] != -1  and  nbor[cen,1] != -1  and  nbor[cen,2] == -1  and  nbor[cen,3] != -1:
        T = torch.einsum('ia,ic,di->acd',right,up,down)
    if nbor[cen,0] != -1  and  nbor[cen,1] != -1  and  nbor[cen,2] != -1  and  nbor[cen,3] == -1:
        T = torch.einsum('ia,bi,ic->acb',right,left,up)
    # two bonds
    if nbor[cen,0] != -1  and  nbor[cen,1] != -1  and  nbor[cen,2] == -1  and  nbor[cen,3] == -1:
        T = torch.einsum('ia,ib->ab',right,up)
    if nbor[cen,0] == -1  and  nbor[cen,1] != -1  and  nbor[cen,2] != -1  and  nbor[cen,3] == -1:
        T = torch.einsum('ai,ib->ba',left,up)
    if nbor[cen,0] != -1  and  nbor[cen,1] == -1  and  nbor[cen,2] == -1  and  nbor[cen,3] != -1:
        T = torch.einsum('ia,bi->ab',right,down)
    if nbor[cen,0] == -1  and  nbor[cen,1] == -1  and  nbor[cen,2] != -1  and  nbor[cen,3] != -1:
        T = torch.einsum('ai,bi->ab',left,down)
    # 缩并的指标根据图形判断，也和b矩阵有关，判断矩阵的行和列对应的棒即可
    # 缩并时，缩并掉公用棒
    # 至于所得张量元，可以自行设置棒标次序
    return T

tensors = []
for i in range(Lx*Ly):
    T = calculate_tensor( B_list, Lx, Ly, i, nbor, mydtype=torch.float64, mydevice=torch.device('cpu'))
    tensors.append(T)

tensors2 = []
for i in range(Lx*Ly):
    iy = int((i)/Lx)
    ix = i-(iy)*Lx
    A2 = tensors[i]
    A3 = tensors[i]
    A4 = tensors[i]
    if ix == 0:
        if iy == 0:
            tensors2.append([A2[:,:,None]])
        elif iy == Ly-1:
            tensors2.append([A2[:,None,:]])
        else:
            tensors2.append([A3[:,:,None,:]])  
    elif ix == Lx-1:
        if iy == 0:
            tensors2.append([A2[None,:,:]])
        elif iy == Ly-1:
            tensors2.append([A2[None,:,:]])
        else:
            tensors2.append([A3[None,:,:,:]])    
    else:
        if iy == 0:
            tensors2.append([A3])
        elif iy == Ly-1:
            tensors2.append([A3])
        else:
            tensors2.append([A4])

tensors = np.array(tensors2).reshape(Ly,Lx)
tensors_list = tensors
# print('================================= tensors list =========================================================')
display('tensors list = ',tensors)

'tensors list = '

array([[tensor([[[ 2.2950],
                 [ 0.1518]],

                [[-0.1535],
                 [ 1.0965]]], dtype=torch.float64),
        tensor([[[-2.4482, -0.2523],
                 [ 0.1267,  1.1807]],

                [[-0.1631,  1.1693],
                 [-1.1805,  0.1031]]], dtype=torch.float64),
        tensor([[[-2.4620e+00, -3.8657e-02],
                 [ 1.6962e-03, -1.1822e+00]],

                [[ 3.7676e-01, -1.1587e+00],
                 [-1.1613e+00, -1.6190e-01]]], dtype=torch.float64),
        tensor([[[ 2.2926e+00,  2.2204e-16],
                 [ 1.1102e-16, -1.1079e+00]]], dtype=torch.float64)],
       [tensor([[[[-2.5289,  0.1244]],

                 [[ 0.2041, -1.1439]]],


                [[[ 0.2448, -1.1424]],

                 [[-1.1426, -0.2775]]]], dtype=torch.float64),
        tensor([[[[ 2.6370,  0.3500],
                  [ 0.0076, -1.2480]],

                 [[-0.0361, -1.2506],
                  [ 1.2616, -0.1462]]],


                [[[-0.30

# (3)缩并张量网络
* Indexing of tensors are clock-wise
<img src="./asset/image.png" width="500">

In [60]:
import math
import sys

def eat(mps,mpo):
    t=[]
    for i in range(len(mps)):
        t.append(torch.einsum("ijk,abcj->aibck",mps[i],mpo[i]).contiguous().view( -1, 2, mps[i].shape[2]*mpo[i].shape[2]))
    return t

def eat2(mps1,mps2):
    t=[]
    for i in range(len(mps1)):
        t.append(torch.einsum("ijk,abj->aibk",mps1[i],mps2[i]).contiguous().view( -1, mps1[i].shape[2] * mps2[i].shape[1]))
    return t

def compress(mps,chi):
    # --------------------------------------------------- QR ---------------------------------------------------
    for i in range(len(mps)-1):
        tensor3 = mps[i].permute(2, 1, 0)
        matrix  = tensor3.contiguous().view( tensor3.shape[0]*2, -1)
        Q, R    = torch.linalg.qr(matrix)
        tensorQ = Q.contiguous().view(tensor3.shape[0], 2, -1)
        mps[i]  = tensorQ.permute(2, 1, 0)
        R       = R.permute(1, 0)
        mps[i+1] = torch.einsum("ij,abi->abj", R, mps[i+1])
    # --------------------------------------------------- SVD ---------------------------------------------------
    for i in range(len(mps)-1,0,-1):
        tensor1 = mps[i-1].permute(2, 1, 0)
        tensor2 = mps[i].permute(2, 1, 0)
        # 合并
        matrix = torch.einsum("ijk,kab->ijab",tensor1,tensor2).view(tensor1.shape[0]*2,tensor2.shape[2]*2)
        # svd
        [U,s,V] = torch.svd(matrix)
        # 构建张量元
        tensor2  = V[:,:chi].t().contiguous().view(-1,2,tensor2.shape[2])
        tensor1 = (U[:,:chi]@torch.diag(s[:chi])).contiguous().view(tensor1.shape[0],2,-1)
        mps[i] = tensor2.permute(2, 1, 0)
        mps[i-1] = tensor1.permute(2, 1, 0)
#         tnorm = mps[i-1].norm()
#         mps[i-1] /= tnorm
    return mps

def get_Z(tensors,chi):
    # ---------------------- 缩并 ------------------------------
    mps = tensors[0][:]
    for head in range(Ly-1):
        if head != Ly-2:
            mps = eat( mps, tensors[head+1][:])
            mps = compress( mps, chi)
        else:
            mps = eat2( mps, tensors[head+1][:])
    # ---------------------- Z ---------------------------------
    T = mps[0]
    for i in range(len(mps)-1):
        T = torch.einsum('ij,ki->kj', T, mps[i+1])
    return torch.trace(T)

# main
Z = get_Z(np.copy(tensors_list),chi)
Z="%20.16f"%Z
print(Z)

2982748.3288340847939253
